In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
import nipype.interfaces.mrtrix3 as mtx
import nipype.interfaces.mrtrix.convert as mtxc
import nipype.interfaces.mrtrix.preprocess as mtxp
import nipype.interfaces.fsl as fsl
from pandas import Series, read_csv, to_numeric
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd, makedirs
from shutil import copyfile
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()

In [2]:
#Set user and path variables
local='False'
user = expanduser('~')
if user == '/Users/lucindasisk':
    if local == 'True':
        laptop = '/Users/lucindasisk/Desktop/DATA'
        home = join(user, 'Desktop/Milgram/candlab')
        raw_dir = join(home, 'data/mri/bids_recon/shapes')
        proc_dir = join(home, 'analyses/shapes/dwi')
        workflow_dir = join(laptop, 'workflows_ls')
        data_dir = join(laptop, 'data_ls')
    else:
        home = join(user, 'Desktop/Milgram/candlab')
        raw_dir = join(home, 'data/mri/bids_recon/shapes')
        proc_dir = join(home, 'analyses/shapes/dwi/data')
        workflow_dir = join(home, 'analyses/shapes/dwi/workflows')
        data_dir = join(home, 'analyses/shapes/dwi/data')
else:
    home = '/gpfs/milgram/project/gee_dylan/candlab'
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    proc_dir = join(home, 'analyses/shapes/dwi/data')
    workflow_dir = join(home, 'analyses/shapes/dwi/workflows')
    data_dir = join(home, 'analyses/shapes/dwi/data')
    
# Read in subject subject_list
subject_info = read_csv(
    home + '/scripts/shapes/mri/dwi/shapes_dwi_subjList_08.07.2019.txt', sep=' ', header=None)
subject_list = subject_info[0].tolist()

# Manual subject list
# subject_list = ['sub-A208', 'sub-A207']

In [3]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir,
                        substitutions = [('_subject_id_', '')]),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(dti = join(proc_dir,'3_Eddy_Corrected/{subject_id}/eddy_corrected_resample.nii.gz'),
                bval = join(raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec = join(proc_dir,'3_Eddy_Corrected/{subject_id}/eddy_corrected.eddy_rotated_bvecs'),
                t1 = join(proc_dir, '2_Preprocessed/{subject_id}/{subject_id}_ses-shapesV1_T1w_flirt_brain.nii.gz'),
                mni=join(home, 'atlases/MNI152_T1_2mm_brain.nii.gz')
               )

sf = Node(SelectFiles(template, 
                      base_directory = home),
          name = 'sf')
                

### Nodes for Diffusion workflow

In [6]:
#Generate binary mask
bet=Node(fsl.BET(frac=0.2,
                mask=True),
        name='bet')

#Convert bvals and bvecs to fslgrad
gradconv = Node(mtx.MRConvert(),
               name = 'gradconv')

#Generate 5 tissue type (5tt) segmentation using FAST algorithm
seg5tt = Node(mtx.Generate5tt(algorithm = 'fsl',
                             out_file = 'T1s_5tt_segmented.nii.gz'),
             name='seg5tt')

#Estimate response functions for spherical deconvolution using the specified algorithm (Dhollander)
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/preprocess.html#responsesd
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/response_function_estimation.html#response-function-estimation
#Max_sh (lmax variable) determined in shell order from here: https://mrtrix.readthedocs.io/en/3.0_rc2/constrained_spherical_deconvolution/lmax.html
#DWI has 5 shells: 7 b0 volumes, 6 b500 vols, 15 b1000 vols, 15 b2000 bols, 60 b3000 vols
dwiresp = Node(mtx.ResponseSD(algorithm = 'dhollander',
                              max_sh=[0,2,4,4,8],
                              wm_file = 'wm_response.txt',
                              gm_file = 'gm_response.txt',
                              csf_file = 'csf_response.txt'),
              name='dwiresp')

#Estimate fiber orientation distributions from diffusion data sing spherical deconvolution
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/reconst.html
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/multi_shell_multi_tissue_csd.html
#Max SH here determined by tissue type - chose 8,8,8 per forum recommendations
mscsd = Node(mtx.EstimateFOD(algorithm = 'msmt_csd',
                             bval_scale = 'yes',
                            max_sh = [8,8,8]),
            name='mscsd')

#Perform Tractography - iFOD2 (https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/tracking.html) 
tract = Node(mtx.Tractography(algorithm='iFOD2',
                              select=100000, #Jiook has done 100 million streamlines
                              n_trials=10000, 
                              out_file='msCSD_brain_tracktography.tck'),
            name='tract')

#Perform probabilistic tractography (Tensor_Prob)
tract_prob = Node(mtx.Tractography(algorithm='Tensor_Prob',
                              select=100000, #Jiook has done 100 million streamlines
                              n_trials=10000, 
                              out_file='tensorProb_brain_tracktography.tck'),
                  name='tract_prob')

#Convert whole-brain tractography from MrTrix format to TrackVis
trkconvert = Node(mtxc.MRTrix2TrackVis(out_filename = 'msCSD_tractography_converted.trk'),
                 name='trkconvert')

trkconvert2 = Node(mtxc.MRTrix2TrackVis(out_filename = 'tensorProb_tractography_converted.trk'),
                 name='trkconvert2')

#convert eddy-corrected raw DTI to tensor format
dwi2tensor = Node(mtx.FitTensor(out_file = 'whole_brain_tensorfile.mif',
                               bval_scale='yes'),
                name='dwi2tensor')

#Compute FA from tensor files
tensor2fa = Node(mtx.TensorMetrics(out_fa='whole_brain_FA.mif'),
                name='tensor2fa')

In [7]:
tract_flow = Workflow(name = 'tract_flow')
tract_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                    #Skullstrip T1
                    (sf, bet, [('t1', 'in_file')]),
#                     #Segment T1 image with FSL 5tt algorithm
#                     (sf, seg5tt, [('t1', 'in_file')]),
#                     (seg5tt, datasink, [('out_file', '5_tract_Reconstruction')]),
                    #Convert bval/bvec to gradient tables
                    (sf, gradconv, [('dti', 'in_file'),
                                   ('bval','in_bval'),
                                   ('bvec', 'in_bvec')]),
                    #Compute FOD response functions
                    (gradconv, dwiresp, [('out_file', 'in_file')]),
                    (dwiresp, datasink, [('wm_file', '5_tract_Reconstruction.@par'),
                                        ('gm_file', '5_tract_Reconstruction.@par.@par'),
                                        ('csf_file', '5_tract_Reconstruction.@par.@par.@par')]),
                    (gradconv, mscsd, [('out_file', 'in_file')]),
                    #Perform multi-shell constrained spherical deconvolution
                    (dwiresp, mscsd, [('wm_file', 'wm_txt'),
                                      ('gm_file', 'gm_txt'),
                                      ('csf_file', 'csf_txt')]),
                    (mscsd, tract, [('wm_odf', 'in_file')]),
                    (mscsd, datasink, [('wm_odf', '5_tract_Reconstruction.@par.@par.@par.@par'),
                                       ('gm_odf', '5_tract_Reconstruction.@par.@par.@par.@par.@par'),
                                       ('csf_odf','5_tract_Reconstruction.@par.@par.@par.@par.@par.@par')]),
                    (gradconv, tract, [('out_file', 'in_file')]),
                    (bet, tract, [('mask_file', 'seed_image')]),
                    #Convert ms-csd files to global tractography
                    (tract, trkconvert, [('out_file', 'in_file')]),
                    (sf, trkconvert, [('t1','image_file')]),
                    (sf, trkconvert, [('t1','registration_image_file')]),
                    (trkconvert, datasink, [('out_file', '5_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par')]),
                    (tract, datasink, [('out_file', '5_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par.@par')]),      
                    (bet, datasink, [('mask_file','5_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),
                    (gradconv, tract_prob, [('out_file', 'in_file')]),
                    (bet, tract_prob, [('mask_file', 'seed_image')]),
                    (tract_prob, datasink, [('out_file','5_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),   
                    (tract_prob, trkconvert2, [('out_file', 'in_file')]),
                    (sf, trkconvert2, [('t1','image_file')]),
                    (sf, trkconvert2, [('t1','registration_image_file')]),
                    (trkconvert2, datasink, [('out_file', '5_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),   
                    #Nodes to create tensor FA files
                    (gradconv, dwi2tensor, [('out_file', 'in_file')]),
                    (dwi2tensor, datasink, [('out_file', '6_Tensor_Data')]),
                    (dwi2tensor, tensor2fa, [('out_file', 'in_file')]),
                    (tensor2fa, datasink, [('out_fa', '6_Tensor_Data.@par')]),
                   ])
tract_flow.base_dir = workflow_dir
tract_flow.write_graph(graph2use = 'flat')
dwi = tract_flow.run('MultiProc', plugin_args={'n_procs': 4})

191031-12:15:24,796 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): No edge data
191031-12:15:24,796 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
191031-12:15:24,797 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.bet): No edge data
191031-12:15:24,798 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.bet): new edge data: {'connect': [('t1', 'in_file')]}
191031-12:15:24,799 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.gradconv): No edge data
191031-12:15:24,800 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.gradconv): new edge data: {'connect': [('dti', 'in_file'), ('bval', 'in_bval'), ('bvec', 'in_bvec')]}
191031-12:15:24,800 nipype.workflow DEBUG:
	 (tract_flow.gradconv, tract_flow.dwiresp): No edge data
191031-12:15:24,801 nipype.workflow DEBUG:
	 (tract_flow.gradconv, tract_flow.dwiresp): new edge data: {'connect': [('out_file', 'in_file')]}
191031-12:15:24,801 nipype.w

191031-12:15:25,656 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A202
191031-12:15:25,659 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A204
191031-12:15:25,660 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A204
191031-12:15:25,665 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A206
191031-12:15:25,665 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A206
191031-12:15:25,668 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A207
191031-12:15:25,669 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A207
191031-12:15:25,673 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A208
191031-12:15:25,674 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A208
191031-12:15:25,678 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A210
191031-12:15:25,679 nipype.wor

191031-12:15:26,82 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A583
191031-12:15:26,86 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A585
191031-12:15:26,87 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A585
191031-12:15:26,91 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A587
191031-12:15:26,92 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A587
191031-12:15:26,95 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A591
191031-12:15:26,96 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A591
191031-12:15:26,100 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A592
191031-12:15:26,101 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A592
191031-12:15:26,105 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A593
191031-12:15:26,106 nipype.workflow D

191031-12:15:26,280 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A665
191031-12:15:26,281 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A665
191031-12:15:26,285 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A666
191031-12:15:26,286 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A666
191031-12:15:26,289 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A671
191031-12:15:26,290 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A671
191031-12:15:26,293 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A675
191031-12:15:26,293 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A675
191031-12:15:26,296 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = sub-A677
191031-12:15:26,298 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A677
191031-12:15:26,302 nipype.wor

191031-12:15:26,385 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A637
191031-12:15:26,386 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,387 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A636
191031-12:15:26,387 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,387 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A634
191031-12:15:26,388 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,388 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A633
191031-12:15:26,389 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,389 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A632
191031-12:15:26,390 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from t

191031-12:15:26,423 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A417
191031-12:15:26,424 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,425 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A415
191031-12:15:26,425 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,425 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A401
191031-12:15:26,426 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,426 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A383
191031-12:15:26,426 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,427 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A374
191031-12:15:26,427 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from t

191031-12:15:26,463 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A240
191031-12:15:26,464 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,464 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A238
191031-12:15:26,471 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,472 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A237
191031-12:15:26,472 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,473 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A236
191031-12:15:26,474 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from the graph.
191031-12:15:26,475 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A235
191031-12:15:26,475 nipype.workflow DEBUG:
	 Removed the identity node tract_flow.infosource from t

KeyboardInterrupt: 

In [ ]:
# import os.path as op
# import matplotlib.pyplot as plt
# import numpy as np
# import nibabel as nib
# import dipy.data as dpd
# from dipy.data import fetcher
# import dipy.tracking.utils as dtu
# import dipy.tracking.streamline as dts
# from dipy.io.streamline import save_tractogram, load_tractogram
# from dipy.stats.analysis import afq_profile, gaussian_weights
# from dipy.io.stateful_tractogram import StatefulTractogram
# from dipy.io.stateful_tractogram import Space

# import AFQ.utils.streamlines as aus
# import AFQ.data as afd
# import AFQ.tractography as aft
# import AFQ.registration as reg
# import AFQ.dti as dti
# import AFQ.segmentation as seg
# from AFQ.utils.volume import patch_up_roi

In [ ]:
# Create Nodes for AFQ Tractography Workflow
# Load processed data



In [ ]:
# origin = '/Users/lucindasisk/Desktop/DATA/tractography_data/4_tract_Reconstruction/sub-A200'
# bval = join(raw_dir, 'sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.bval')
# bvec = join(proc_dir,'eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.eddy_rotated_bvecs')
# dtifile = join(proc_dir,'eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.nii.gz')
# img = nib.load(proc_dir +'/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.nii.gz')

# print("Calculating DTI...")
# if not op.exists(origin + '/dti_FA.nii.gz'):
#     dti_params = dti.fit_dti(dtifile, bval, bvec,
#                              out_dir=origin)
# else:
#     dti_params = {'FA': './dti_FA.nii.gz',
#                   'params': './dti_params.nii.gz'}
# FA_img = nib.load(dti_params['FA'])
# FA_data = FA_img.get_fdata()

# tg = load_tractogram(origin + '/whole_brain_tractography_converted.trk', img)
# streamlines = tg.streamlines

# # streamlines = dts.Streamlines(
# #     dtu.transform_tracking_output(streamlines,
# #                                   np.linalg.inv(img.affine)))

# templates = afd.read_templates()
# bundle_names = ["UNC", "CGC"]

# bundles = {}
# for name in bundle_names:
#     for hemi in ['_R', '_L']:
#         bundles[name + hemi] = {
#             'ROIs': [templates[name + '_roi1' + hemi],
#                      templates[name + '_roi2' + hemi]],
#             'rules': [True, True],
#             'prob_map': templates[name + hemi + '_prob_map'],
#             'cross_midline': False}

# print("Registering to template...")
# MNI_T2_img = dpd.read_mni_template()
# if not op.exists('mapping.nii.gz'):
#     import dipy.core.gradients as dpg
#     gtab = dpg.gradient_table(bval, bvec)
#     warped_hardi, mapping = reg.syn_register_dwi(dtifile, gtab)
#     reg.write_mapping(mapping, './mapping.nii.gz')
# else:
#     mapping = reg.read_mapping('./mapping.nii.gz', img, MNI_T2_img)

# tg = load_tractogram('/Users/lucindasisk/Desktop/DATA/tractography_data/4_tract_Reconstruction/sub-A200/whole_brain_tractography_converted.trk', img)     
# streamlines = tg.streamlines

# streamlines = dts.Streamlines(
# dtu.transform_tracking_output(streamlines,
#                               np.linalg.inv(img.affine)))

# print("Segmenting fiber groups...")
# segmentation = seg.Segmentation()
# segmentation.segment(bundles,
#                      streamlines,
#                      fdata=dti,
#                      fbval=bval,
#                      fbvec=bvec,
#                      mapping=mapping,
#                      reg_template=MNI_T2_img)


# fiber_groups = segmentation.fiber_groups

# print("Cleaning fiber groups...")
# for bundle in bundles:
#     fiber_groups[bundle] = seg.clean_fiber_group(fiber_groups[bundle])

# for kk in fiber_groups:
#     print(kk, len(fiber_groups[kk]))

#     sft = StatefulTractogram(
#         dtu.transform_tracking_output(fiber_groups[kk], img.affine),
#         img, Space.RASMM)

#     save_tractogram(sft, origin + '/%s_afq.trk'%kk,
#                     bbox_valid_check=False)

# print("Extracting tract profiles...")
# for bundle in bundles:
#     fig, ax = plt.subplots(1)
#     weights = gaussian_weights(fiber_groups[bundle])
#     profile = afq_profile(FA_data, fiber_groups[bundle],
#                           np.eye(4), weights=weights)
#     ax.plot(profile)
#     ax.set_title(bundle)

# plt.show()

In [ ]:
# (mscsd, datasink, [('wm_odf', '4_tract_Reconstruction.@par.@par.@par.@par'),
#                                        ('gm_odf', '4_tract_Reconstruction.@par.@par.@par.@par.@par'),
#                                        ('csf_odf','4_tract_Reconstruction.@par.@par.@par.@par.@par.@par')]),


# dwi_flow = Workflow(name = 'dwi_flow')
# dwi_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
#                   (sf, mrconv, [('dti', 'in_file')]),
#                   (mrconv, dwi_res, [('out_file', 'in_file')]),
#                   (sf, dwi_res, [('bval', 'in_bval'),
#                                  ('bvec', 'in_bvec')]),
#                   (dwi_res, datasink, [('csf_file', '4_DWI_Reconstruction.@par'),
#                                       ('wm_file', '4_DWI_Reconstruction.@par.@par'),
#                                       ('gm_file', '4_DWI_Reconstruction.@par.@par.@par')]),
#                   (sf, ms_csd, [('dti','in_file'),
#                                ('bval','in_bval'),
#                                ('bvec','in_bvec'),
#                                ('mask', 'mask_file')]),
#                   (dwi_res, ms_csd, [('wm_file', 'wm_txt'),
#                                      ('gm_file', 'gm_txt'),
#                                      ('csf_file', 'csf_txt')]),
#                   (ms_csd, datasink, [('wm_odf', '4_DWI_Reconstruction.@par.@par.@par.@par'),
#                                      ('gm_odf','4_DWI_Reconstruction.@par.@par.@par.@par.@par'),
#                                      ('csf_odf', '4_DWI_Reconstruction.@par.@par.@par.@par.@par.@par')]) 
#                  ])
# dwi_flow.base_dir = workflow_dir
# dwi_flow.write_graph(graph2use = 'flat')
# dwi = dwi_flow.run('MultiProc', plugin_args={'n_procs': 4})